In [8]:
import numpy as np
import pynq
from pynq import MMIO
import cv2
import random
import time

in1 = np.loadtxt("in1.txt", dtype=int)
in2 = np.loadtxt("in2.txt", dtype=int)
in1 = in1.reshape(100 * 100)
in2 = in2.reshape(100 * 100)


overlay = pynq.Overlay("design_1.bit")
top_ip = overlay.ref_LK_0
top_ip.signature

inp1_buffer = pynq.allocate((100 * 100), np.int16)
inp2_buffer = pynq.allocate((100 * 100), np.int16)
op1_buffer = pynq.allocate((100 * 100), np.int16)
op2_buffer = pynq.allocate((100 * 100), np.int16)
start = time.time()
for i, pixel in enumerate(in1):
    inp1_buffer[i] = pixel
for i, pixel in enumerate(in2):
    inp2_buffer[i] = pixel

aptr = inp1_buffer.physical_address
bptr = inp2_buffer.physical_address
cptr = op1_buffer.physical_address
dptr = op2_buffer.physical_address

top_ip.write(0x18, aptr)
top_ip.write(0x24, bptr)
top_ip.write(0x30, cptr)
top_ip.write(0x3C, dptr)
top_ip.write(0x00, 1)

isready = top_ip.read(0x00)
end = time.time()
while isready == 1:
    isready = top_ip.read(0x00)

np.savetxt("fpga_x.txt", np.asarray(op1_buffer), fmt="%10.10f", newline=",")
np.savetxt("fpga_y.txt", np.asarray(op2_buffer), fmt="%10.10f", newline=",")

print("Time to run on FPGA: ", end - start)

Time to run on FPGA:  0.08475255966186523


(10000,)


0.24912166595458984


1.5360655784606934


In [58]:
print("InputImage1:")
print(inp1_buffer[0:1000])

print("InputImage2:")
print(inp2_buffer[0:1000])


print("\nFPGA returns:")
print(op1_buffer[1000:2000])

print("\nFPGA returns:")
print(op2_buffer[1000:2000])

InputImage1:
[114  68 173 122  66  67 168 112  39  79 174 125  53  88 169 108  62  62
 169 127  91  72 168 115 134 129 207 184  80 100 174 121  62  80 169 114
  75  85 194 188 144 100 182 139 150  92 189 163 123  81 173 116 109 170
 187 124  64  94 181 137  65  80 177 133 117  76 174 125  55 117 190 161
  98 133 193 200 138  91 201 232 193 143 187 177 141  95 199 216 112 130
 188 176 139  97 204 224 147  76 204 236  72 159 189 138  92 176 202 186
 189 174 219 198 253 191 228 217 169 206 187 111  53  87 210 228 166 185
 215 198 205 170 239 254  37 144 201 177  82  76 165 115 163 106 182 157
 116 174 211 246 230 146 192 186 147 105 179 146 110  75 188 170 232 139
 189 157 106 112 185 166  78  85 194 198 124 104 191 163 121  86 172 131
 105 126 179 145 117  92 184 170  83 143 206 213  79  76 159  92  66  81
 187 172 134 106 176 143  91  79 168 127 166 132 196 203 181 142 195 211
 147 105 183 186 110 157 185 148 112  99 208 211 121  95 186 181  97 105
 180 149  79 113 179 152  66  80 184 1

In [70]:
np.savetxt("fpga_x.txt", np.asarray(op1_buffer), fmt="%10.10f", newline=",")
np.savetxt("fpga_y.txt", np.asarray(op2_buffer), fmt="%10.10f", newline=",")